In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from state_cleaner import *
from datetime import datetime
import requests
import json

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile'

In [33]:
fips_to_county = pd.read_csv('./data/ZIP-COUNTY-FIPS_2017-06.csv',converters={'STCOUNTYFP': str})
full_data = pd.read_csv('./data/compiled_2020-05-22.csv',converters={'FIPS': str})

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (104,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,139,140,143,144) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
full_data[['FIPS','park']][full_data['park'].notna()]

,FIPS,park
1,1001.0,-23.0
36,1001.0,-14.0
57,1001.0,-34.0
64,1001.0,-65.0
93,1003.0,55.0
94,1003.0,-12.0
95,1003.0,31.0
96,1003.0,21.0
97,1003.0,41.0
98,1003.0,-10.0


In [23]:
fips = parks['FIPS'].unique()
fips = fips[1:]
len(fips)

955

In [25]:
fips

array(['1001.0', '1003.0', '1015.0', '1033.0', '1043.0', '1049.0',
       '1051.0', '1055.0', '1069.0', '1073.0', '1077.0', '1081.0',
       '1083.0', '1089.0', '1095.0', '1097.0', '1101.0', '1103.0',
       '1117.0', '1121.0', '1125.0', '2020.0', '2110.0', '4001.0',
       '4003.0', '4005.0', '4007.0', '4009.0', '4012.0', '4013.0',
       '4015.0', '4017.0', '4019.0', '4021.0', '4023.0', '4025.0',
       '4027.0', '5007.0', '5031.0', '5045.0', '5051.0', '5071.0',
       '5091.0', '5119.0', '5125.0', '5131.0', '5143.0', '5145.0',
       '5149.0', '6001.0', '6005.0', '6007.0', '6009.0', '6013.0',
       '6015.0', '6017.0', '6019.0', '6021.0', '6023.0', '6025.0',
       '6027.0', '6029.0', '6031.0', '6033.0', '6037.0', '6039.0',
       '6041.0', '6043.0', '6045.0', '6047.0', '6051.0', '6053.0',
       '6055.0', '6057.0', '6059.0', '6061.0', '6065.0', '6067.0',
       '6069.0', '6071.0', '6073.0', '6075.0', '6077.0', '6079.0',
       '6081.0', '6083.0', '6085.0', '6087.0', '6089.0', '6093

In [24]:
fips_to_zips = dict()
for fip in fips:
    fips_to_zips[int(fip)] = list(fips_to_county[fips_to_county['STCOUNTYFP']==fip]['ZIP'])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


ValueError: invalid literal for int() with base 10: '1001.0'

In [6]:
def get_weather(locationid, datasetid, begin_date, end_date, mytoken, base_url):
    token = {'token': mytoken}
    
    #passing as string instead of dict because NOAA API does not like percent encoding
    params = 'datasetid='+str(datasetid)+'&'+'locationid='+str(locationid)+'&'+'startdate='+str(begin_date)+'&'+'enddate='+str(end_date)+'&'+'limit=25'+'&datatypeid=TOBS,PRCP'
    
    r = requests.get(base_url+params, headers=token)
    #print("Request status code: "+str(r.status_code))

    try:
        #results comes in json form. Convert to dataframe
        df = pd.DataFrame.from_dict(r.json()['results'])
        #print("Successfully retrieved "+str(len(df['station'].unique()))+" stations")
        dates = pd.to_datetime(df['date'])

        return df
    except:
        pass

In [7]:
def already_exists(fip):
    #Checks to see if the fip data is already in the database
    if conn.noaaweather.records.count_documents({ 'FIPS': fip }, limit = 1) != 0:
        return True
    else:
        return False

In [ ]:
import itertools
z = dict(itertools.islice(fips_to_zips.items(), 5))

In [3]:
import json
import pymongo
import time
from pprint import pprint

conn=pymongo.MongoClient()
db = conn.noaaweather
records = db.records
conn.list_database_names()

['admin', 'config', 'local', 'mydb', 'noaaweather']

In [14]:
records.find_one()

{'FIPS': 1001,
 '_id': ObjectId('5eb444b4f987a45eeb9d9046'),
 'date': '2020-03-01T00:00:00',
 'precipitation': 0.0,
 'temp': 33.0}

In [15]:
mar_one = records.find( {'date':'2020-03-01T00:00:00'} )

In [16]:
mar_one.count()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


565

In [ ]:
for doc in mar_one:
    print(doc)

In [9]:
empty_fips = [9001,9003,9005,9007,9009,9011,9013,9015,22075,22095,23001,23003,23005,23007,
              23009,23011,23017,23019,23025,23031,25001,25003,25005,25009,25011,25013,25015,
              25017,25021,25023,25025,25027,26025,39133,39141,39151,39153,39155,39165,39173,
              40017,40021,40027,40031,40037,40047,40049,40089,40099,40101,40109,40113,40119,
              40121,40125,40131,40143,40151,41003,41005,41007,41009,41011,41015,41017,41019,
              41027,41029,41033,41035,41039,41041,41043,41047,41051,41053,41057,41059,41065,
              41067,41071,42001,42003,42007,42011,42013,42017,42019,42025,42027,42029,42033,
              42041,42043,42045,42049,42051,42055,42069,42071,42073,42075,42077,42079,42081,
              42089,42091,42095,42101,42107,42111,42115,42123,42125]

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-02-15'
enddate = '2020-05-17'

tic = time.perf_counter()
weather = pd.DataFrame()
for fip, codes in fips_to_zips.items():
    if not already_exists(fip) and fip not in empty_fips: #check to make sure not double counting data
        print(fip)
        fip_weather = pd.DataFrame()
        for code in codes:
            try:
                newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
                fip_weather = fip_weather.append(newdf)
            except:
                continue
        if not fip_weather.empty:
            #average across the county
            precip = fip_weather[fip_weather['datatype']=='PRCP'][['date','value']].rename(columns={'value':'precipitation'})
            temp = fip_weather[fip_weather['datatype']=='TOBS'][['date','value']].rename(columns={'value':'temp'})
            w = pd.merge(precip, temp, how ='outer', on ='date') 
            avgs = pd.merge(w.groupby('date', as_index=False)['precipitation'].mean(),
                            w.groupby('date', as_index=False)['temp'].mean(),on='date')
            avgs['FIPS']=fip
            avgs = avgs.to_dict()

            #insert records into MongoDB
            a = [{key:value[index] for key,value in avgs.items()}
                 for index in range(max(map(len,avgs.values())))]
            records.insert_many(a)
            print(records.count())
        else:
            empty_fips.append(fip)
    
toc= time.perf_counter()
print(f"Run time was {toc - tic:0.4f} seconds")

33001
33003
33005
33007
33009
33011
33013
33015
33017
34001
34003
34005
34007


In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-03-01'
enddate = '2020-05-01'
newdf = get_weather('ZIP:'+str(6673), databaseid,begindate,enddate,mytoken,baseurl)

In [ ]:
codes = fips_to_zips[6053]
for code in codes:
    try:
        newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
        fip_weather = fip_weather.append(newdf)
    except:
        continue

#### Practice- only with DC zip codes:

In [ ]:
dc_zips = fips_to_zips['District of Columbia']
len(dc_zips)

In [ ]:
baseurl = 'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?'
mytoken = 'NVPWXeGxislWKqThiizoffNjnPtUtAcT'
databaseid = 'GHCND'
begindate = '2020-03-01'
enddate = '2020-05-01'
dc_weather = pd.DataFrame()
for code in dc_zips[0:100]:
    print(code)
    try:
        newdf = get_weather('ZIP:'+str(code), databaseid,begindate,enddate,mytoken,baseurl)
        newdf.insert(0,'ZIP',code)
        dc_weather = dc_weather.append(newdf)
    except:
        continue

In [ ]:
precip = dc_weather[dc_weather['datatype']=='PRCP'][['ZIP','date','value']].rename(columns={'value':'precipitation'})
temp = dc_weather[dc_weather['datatype']=='TOBS'][['date','value']].rename(columns={'value':'temp'})
w = pd.merge(precip, temp, how ='outer', on ='date') 
avgs = pd.merge(w.groupby('date', as_index=False)['precipitation'].mean(),
                w.groupby('date', as_index=False)['temp'].mean(), on='date')
avgs ['County']='DC'
avgs.to_dict()

In [ ]:
ld = [{key:value[index] for key,value in a.items()}
         for index in range(max(map(len,a.values())))]
ld

#### Check whether weather from one ZIP is different across a county

In [ ]:
import scipy.stats as stats

In [ ]:
stats.f_oneway(weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[0]],
               weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[1]],
               weather['precipitation'][weather['ZIP'] == weather['ZIP'].unique()[2]])

In [ ]:
stats.f_oneway(weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[0]].dropna(), 
             weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[1]].dropna(),
             weather['temp'][weather['ZIP'] == weather['ZIP'].unique()[2]].dropna())

### Getting weather data using the noaa package from github:

In [ ]:
n = noaa.NOAA()

In [ ]:
forecast = n.get_observations(20001, 'US',start='2020-04-28',end='2020-04-30')
f = pd.DataFrame(forecast)
items = ['timestamp','textDescription','temperature','precipitationLast6Hours','relativeHumidity','cloudLayers']
f = f[items]
f.insert(0,'ZIP',zip_code) #add zip code 

In [ ]:
dc_weather = pd.DataFrame(columns = ['ZIP','timestamp','textDescription','temperature','precipitationLast6Hours','relativeHumidity','cloudLayers'])
dczips = [20001,20004, 20005, 20036, 20009, 20007, 20010, 20008, 20016]
for code in dczips:
    forecast = n.get_observations(code, 'US',start='2020-04-28',end='2020-04-30')
    for obs in forecast:
        time = obs['timestamp']
        desc = obs['textDescription']
        temp = obs['temperature']['value']
        precip = obs['precipitationLast6Hours']['value']
        hum = obs['relativeHumidity']['value']
        cloud = obs['cloudLayers'][0]['base']['value']
    
        row = [code,time,desc,temp,precip,hum,cloud]
        weather.loc[len(weather)] = row

In [ ]:
weather.to_csv('/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/data/DC_weather_04_28-04_30.csv')

In [ ]:
weather = pd.DataFrame()
bad_zips = []
for zip_code in fips_to_county['ZIP'][0:200]:
    if zip_code not in bad_zips:
        print(zip_code)
        try:
            forecast = n.get_forecasts(zip_code, 'US',hourly = False)
            f = pd.DataFrame(forecast)
            f = f[['startTime','endTime','temperature','windSpeed','windDirection','shortForecast']]
            f.insert(0,'ZIP',zip_code) #add zip code 
            weather = pd.concat([weather, f], ignore_index = True)
        except:
            bad_zips.append(zip_code)
            continue